In [ ]:
import json
import requests
import time
import os

In [ ]:
OPENSEA_API_BASE = "https://api.opensea.io/api/v2"
OPENSEA_API_KEY = "" 
OPENSEA_API_BASE_V1 = "https://api.opensea.io/api/v1" 
collection_slug="doodles-official" # pudgypenguins, chimpersnft,doodles-official
chain="ethereum"

In [ ]:
get_nft_trait_url = "https://api.opensea.io/api/v2/chain/{chain}/contract/{contract_address}/nfts/{identifier}"


In [ ]:
def make_api_request(url, headers=None, params=None):
    """Make API request with basic rate limiting"""
    if headers is None:
        headers = {
            "Accept": "application/json",
            "X-API-KEY": OPENSEA_API_KEY
        }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        
        # Handle rate limiting
        if response.status_code == 429:
            print("Rate limited, waiting 1 second...")
            time.sleep(1)
            return make_api_request(url, headers, params)
        
        # Raise exception for other errors
        response.raise_for_status()
        
        return response.json()
    except Exception as e:
        print(f"Error making request to {url}: {e}")
        return None

In [ ]:
def get_nfts_traits(chain, contract_address, identifier):
    """
    Get NFT
    Ouptut:
        {
  "nft": {
    "identifier": "string",
    "collection": "string",
    "contract": "string",
    "token_standard": "string",
    "name": "string",
    "description": "string",
    "image_url": "",
    "display_image_url": "",
    "display_animation_url": "",
    "metadata_url": "",
    "opensea_url": "",
    "updated_at": "string",
    "is_disabled": true,
    "is_nsfw": true,
    "animation_url": "",
    "is_suspicious": true,
    "creator": "string",
    "traits": [
      {
        "trait_type": "string",
        "display_type": "number",
        "max_value": "string",
        "value": 0
      }
    ],
    "owners": [
      {
        "address": "string",
        "quantity": 0
      }
    ],
    "rarity": {
      "strategy_version": "string",
      "rank": 0,
      "score": 0,
      "calculated_at": "",
      "max_rank": 0,
      "total_supply": 0,
      "ranking_features": {
        "unique_attribute_count": 0
      }
    }
  }
}
    """
    url = get_nft_trait_url.format(chain=chain,contract_address=contract_address, identifier=identifier)
    return make_api_request(url)

In [ ]:
# Get all NFTs in the collection
nfts_by_collections_file = f"./data/{collection_slug}/nfts_by_collection_data_sorted.json"
nfts_traits_output_file = f"./data/{collection_slug}/nfts_traits_data_sorted.json"

nfts_traits_json=[]

with open(nfts_by_collections_file, 'r') as f:
    nfts_by_collections_json = json.load(f)
    for nft in nfts_by_collections_json:
        contract_address = nft['contract']
        identifier = nft['identifier']
        print(f"Getting traits for {contract_address} - {identifier}")
        nft_traits_dic = get_nfts_traits(chain, contract_address, identifier)
        if nft_traits_dic:
            nfts_traits_json.append(nft_traits_dic)
        with open(nfts_traits_output_file, 'w') as f:
            json.dump(nfts_traits_json, f, indent=4)



## Get Images

In [ ]:
# get image for each nft

from email.mime import image


nfts_by_collections_file = f"./data/{collection_slug}/nfts_by_collection_data_sorted.json"
nfts_image_output_dir= f"./data/{collection_slug}/images/"
os.makedirs(nfts_image_output_dir, exist_ok=True)

with open(nfts_by_collections_file, 'r') as f:
    nfts_by_collections_json = json.load(f)
    for nft in nfts_by_collections_json:
        identifier= nft['identifier']
        image_name= f'{collection_slug}_{identifier}' + ".jpg"
        image_url = nft['image_url']
        try:
            # Get the image data
            response = requests.get(image_url, stream=True, timeout=10)
            response.raise_for_status()  # Raise an error for bad status codes
            
            # Save the image
            image_path = os.path.join(nfts_image_output_dir, image_name)
            with open(image_path, 'wb') as img_file:
                for chunk in response.iter_content(1024):
                    img_file.write(chunk)
            
            print(f"Downloaded image for NFT {identifier}")
            
        except requests.exceptions.RequestException as e:
            print(f"Failed to download image for NFT {identifier}: {e}")
        except Exception as e:
            print(f"Error processing NFT {identifier}: {e}")
        